In [1]:
from data_utils import get_CIFAR10_data
import numpy as np

import keras
import tensorflow as tf
from keras.layers import Input, Conv2D, Dense, Flatten, Add, ZeroPadding2D, add, MaxPool2D
from keras.models import Model
from keras.utils.np_utils import to_categorical 
from tensorflow_weave import *

/Users/Takn/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [4]:
#Load the (preprocessed) CIFAR10 data.

data = get_CIFAR10_data()
for k, v in data.items():
  print('%s: ' % k, v.shape)

X_train:  (49000, 3, 32, 32)
y_train:  (49000,)
X_val:  (1000, 3, 32, 32)
y_val:  (1000,)
X_test:  (1000, 3, 32, 32)
y_test:  (1000,)


In [5]:
filter_size = 3
num_filters = 2
pad_size = filter_size//2
mid_layer = 100


X_train = data['X_train']
Y_train = data['y_train']
X_Val = data['X_val']
Y_Val = data['y_val']

Y_train = to_categorical(Y_train, num_classes=10)
Y_val = to_categorical(Y_Val, num_classes=10)

In [6]:
def build_weave_conv_model(num_filters1, num_filters2):
    inputs = Input(shape=(3,32,32))


    ######### Layer 1
    x_pad = ZeroPadding2D(padding=(pad_size,pad_size))(inputs)


    x_per = Conv2D(num_filters1,
                   kernel_size = (filter_size,filter_size),
                   strides=(1,1),
                   padding='valid',
                   activation='relu')(x_pad)

    x_loc = Conv2D(num_filters1,
                   kernel_size= (filter_size,filter_size),
                   strides=(1,1),
                   padding='valid',
                   activation = 'relu')(x_pad)

    x_zero = ZeroWeave()(x_loc)
    x_weave = ArrayWeave()(x_per)

    x = Add()([x_weave, x_zero])

    x = ZeroPadding2D(padding=(pad_size,pad_size))(x)

    x = Conv2D(num_filters2,
               kernel_size= (filter_size,filter_size),
               strides=(filter_size,filter_size),
               padding='valid',
               activation = 'relu')(x)
    x = MaxPool2D()(x)
#     ###############
#     x_pad = ZeroPadding2D(padding=(pad_size,pad_size))(x)


#     x_per = Conv2D(2*num_filters,
#                    kernel_size = (filter_size,filter_size),
#                    strides=(1,1),
#                    padding='valid',
#                    activation='relu')(x_pad)

#     x_loc = Conv2D(2*num_filters,
#                    kernel_size= (filter_size,filter_size),
#                    strides=(1,1),
#                    padding='valid',
#                    activation = 'relu')(x_pad)

#     x_zero = ZeroWeave()(x_loc)
#     x_weave = ArrayWeave()(x_per)

#     x = Add()([x_weave, x_zero])

#     x = ZeroPadding2D(padding=(pad_size,pad_size))(x)

#     x = Conv2D(2*num_filters,
#                kernel_size= (filter_size,filter_size),
#                strides=(filter_size,filter_size),
#                padding='valid',
#                activation = 'relu')(x)

#     x = MaxPool2D()(x)
    x = Flatten()(x)
    x = Dense(mid_layer, activation = 'relu')(x)
    predictions = Dense(10, activation='softmax')(x)

    # This creates a model that includes
    # the Input layer and three Dense layers
    model = Model(inputs=[inputs], outputs=predictions)
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [7]:
#hist1 = model.fit(X_train, Y_train, epochs = 10, validation_split=.05, verbose=1)

In [8]:
def build_norm_conv_model(num_filters1, num_filters2):
    inputs = Input(shape=(3,32,32))

    ####### Layer 1
    x = ZeroPadding2D(padding=(pad_size,pad_size))(inputs)

    x = Conv2D(num_filters1,
                   kernel_size = (filter_size,filter_size),
                   strides=(1,1),
                   padding='valid',
                   activation='relu')(x)

    x = ZeroPadding2D(padding=(pad_size,pad_size))(x)

    x = Conv2D(num_filters1,
                   kernel_size= (filter_size,filter_size),
                   strides=(1,1),
                   padding='valid',
                   activation = 'relu')(x)

    x = ZeroPadding2D(padding=(pad_size,pad_size))(x)

    x = Conv2D(num_filters2,
               kernel_size= (filter_size,filter_size),
               strides=(1,1),
               padding='valid',
               activation = 'relu')(x)

    x = MaxPool2D()(x)
#     ####### Layer 2
#     x = ZeroPadding2D(padding=(pad_size,pad_size))(x)

#     x = Conv2D(2*num_filters,
#                    kernel_size = (filter_size,filter_size),
#                    strides=(1,1),
#                    padding='valid',
#                    activation='relu')(x)

#     x = ZeroPadding2D(padding=(pad_size,pad_size))(x)

#     x = Conv2D(2*num_filters,
#                    kernel_size= (filter_size,filter_size),
#                    strides=(1,1),
#                    padding='valid',
#                    activation = 'relu')(x)

#     x = ZeroPadding2D(padding=(pad_size,pad_size))(x)

#     x = Conv2D(2*num_filters,
#                kernel_size= (filter_size,filter_size),
#                strides=(1,1),
#                padding='valid',
#                activation = 'relu')(x)

#     x = MaxPool2D()(x)
    x = Flatten()(x)
    x = Dense(mid_layer, activation = 'relu')(x)
    predictions = Dense(10, activation='softmax')(x)

    # This creates a model that includes
    # the Input layer and three Dense layers
    model2 = Model(inputs=[inputs], outputs=predictions)
    model2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model2

In [9]:
#hist2 = model2.fit(X_train, Y_train, epochs = 10, validation_split=.05, verbose=1)

In [ ]:
filters1 = [1,2,4,8,12]
filters2 = [1,2,4,8,12]
num_epochs = 10
weave_acc = {}
norm_acc = {}
for filt1 in filters1:
    for filt2 in filters2:
        print(filt1,filt2)
        weave_model = build_weave_conv_model(filt1,filt2)
        norm_model = build_norm_conv_model(filt1,filt2)
        weave_model.fit(X_train, Y_train, epochs = num_epochs, verbose=0)
        norm_model.fit(X_train, Y_train, epochs = num_epochs, verbose=0)
        weave_acc[(filt1,filt2)] = weave_model.evaluate(X_Val, Y_val, verbose = 0)[1]
        norm_acc[(filt1,filt2)] = norm_model.evaluate(X_Val, Y_val, verbose = 0)[1]
        print('Weave Model', weave_acc[(filt1,filt2)])
        print('Norm Model', norm_acc[(filt1,filt2)])

1 1
Weave Model 0.418
Norm Model 0.438
1 2
Weave Model 0.42
Norm Model 0.518
1 4
Weave Model 0.474
Norm Model 0.535
1 8
Weave Model 0.506
Norm Model 0.498
1 12
Weave Model 0.498
Norm Model 0.541
2 1
Weave Model 0.437
Norm Model 0.444
2 2
Weave Model 0.452
Norm Model 0.494
2 4
Weave Model 0.112
Norm Model 0.516
2 8
Weave Model 0.087
Norm Model 0.564
2 12
Weave Model 0.107
Norm Model 0.502
4 1
